### This program calculates the spaial error and shape error between a reference image and a drawing
- Shape Error: Measured by MSE
- Spatial Error: Rotation, Translation, Scaling

### Basic Setup

#### Load dependencies and define directory hierarchy

In [19]:
import sys
import os

import pymongo as pm ## first establish ssh tunnel to server where database is running
import numpy as np
from numpy import *
import cv2
import pandas as pd
import svg_distance_helpers as rsh
import affine_registration_2d as ar

#### File hierarchy

In [20]:
analysis_dir = os.getcwd()
iteration_name = 'all_tracings'

ref_dir = os.path.join(analysis_dir, 'tracing_ref_400')
data_dir = os.path.join(analysis_dir, 'all_tracings_june_2021')

loss = 'ncc'
trans_dir = os.path.join(analysis_dir, 'transformed_{}'.format(loss)) # store transformed images
if not os.path.exists(trans_dir):
    os.mkdir(trans_dir)

canvas_side = 400 # image size
tracing = ['square','shape']

data_output = 'devphotodraw_tracing_{}.csv'.format(iteration_name, loss)

#### Check how many items get transformed

In [21]:
# # get data from csv file
# data = pd.read_csv(data_output)
# try:
#     data = data.drop("Unnamed: 0", axis=1)
# except:
#     print "Already droped"
# print data.count()

#### Generate binary reference shapes

In [22]:
really_run_this = 1
ref_path = []
dim = (424,424)
if really_run_this:
    for t in tracing:
        fname = os.path.join(ref_dir, '{}.png'.format(t))
        img_ref = cv2.imread(fname)
        img_ref = cv2.resize(img_ref, dim)
        img_ref = rsh.color_to_num(img_ref, threshold=0, white_background=False)
        ofname = os.path.join(ref_dir, '{}_ref.png'.format(t))
        cv2.imwrite(ofname, 255-img_ref*255)

In [23]:
ref_path = []
for t in tracing:
    ofname = os.path.join(ref_dir, '{}_ref.png'.format(t))
    ref_path.append(ofname)
ref_path

['/Users/brialong/Documents/GitHub/devphotodraw/analysis/step5_tracing_evaluation/step1_render_highres_tracings/tracing_ref_400/square_ref.png',
 '/Users/brialong/Documents/GitHub/devphotodraw/analysis/step5_tracing_evaluation/step1_render_highres_tracings/tracing_ref_400/shape_ref.png']

### Calculate Shape Error and Spatial Error

In [24]:
reload(ar)
filename = []
category =[] 
pre_tran = []
post_tran = []
rotate = []
translate = []
scale = []

In [25]:
# d = 'Tsinghua_photodraw_production1575443112866_shape.png'
# test = d.split('_')
# stripped_filename = d.split('.png')[0]
# stripped_filename

In [ ]:
for ti, t in enumerate(tracing):
    t_dir = os.path.join(data_dir, t)
    count_done = 0
    for d in os.listdir(t_dir):
        count_done = count_done + 1
        print('getting tracing for {} of {} for {}'.format(count_done, len(os.listdir(t_dir)),t))
        if not d.endswith('.png'):
            continue
        
        # Tsinghua_photodraw_production1575443112866_shape
        # CDM_photodraw_e21542061442525_square.png
        
        child_info = d.split('_')
        stripped_filename = d.split('.png')[0]
        img_draw = os.path.join(t_dir, d)

        if os.path.exists(img_draw):
            img_ref = ref_path[ti]
            output_path = os.path.join(trans_dir, '{}_wrap.png'.format(d))
            init_loss, final_loss, ro, tran, s, warped = ar.affine_reg(img_draw, img_ref, output_path)

            # prepare the dataframe
            filename.append(d)
            category.append(t)
            pre_tran.append(init_loss)
            post_tran.append(final_loss)
            rotate.append(ro)
            translate.append(tran)
            scale.append(s)

getting tracing for 1 of 249 for square
getting tracing for 2 of 249 for square
getting tracing for 3 of 249 for square
getting tracing for 4 of 249 for square
getting tracing for 5 of 249 for square
getting tracing for 6 of 249 for square
getting tracing for 7 of 249 for square
getting tracing for 8 of 249 for square
getting tracing for 9 of 249 for square
getting tracing for 10 of 249 for square
getting tracing for 11 of 249 for square
getting tracing for 12 of 249 for square
getting tracing for 13 of 249 for square
getting tracing for 14 of 249 for square
getting tracing for 15 of 249 for square
getting tracing for 16 of 249 for square
getting tracing for 17 of 249 for square
getting tracing for 18 of 249 for square
getting tracing for 19 of 249 for square
getting tracing for 20 of 249 for square
getting tracing for 21 of 249 for square
getting tracing for 22 of 249 for square
getting tracing for 23 of 249 for square
getting tracing for 24 of 249 for square
getting tracing for 25 of

getting tracing for 199 of 249 for square
getting tracing for 200 of 249 for square
getting tracing for 201 of 249 for square
getting tracing for 202 of 249 for square
getting tracing for 203 of 249 for square
getting tracing for 204 of 249 for square
getting tracing for 205 of 249 for square
getting tracing for 206 of 249 for square
getting tracing for 207 of 249 for square
getting tracing for 208 of 249 for square
getting tracing for 209 of 249 for square
getting tracing for 210 of 249 for square
getting tracing for 211 of 249 for square
getting tracing for 212 of 249 for square
getting tracing for 213 of 249 for square
getting tracing for 214 of 249 for square
getting tracing for 215 of 249 for square
getting tracing for 216 of 249 for square
getting tracing for 217 of 249 for square
getting tracing for 218 of 249 for square
getting tracing for 219 of 249 for square
getting tracing for 220 of 249 for square
getting tracing for 221 of 249 for square
getting tracing for 222 of 249 for

In [27]:
really_run_this = 1 
## compile data
if really_run_this==1:
    data = pd.DataFrame([filename, category, pre_tran, post_tran, rotate, translate, scale])
    data = data.transpose()
    data.columns = ['filename','category', 'pre_tran', 'post_tran', 'rotate', 'translate', 'scale']
## save out the data
data.to_csv(data_output)